<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/Practica1/3_2_scrapyJSON_LD_digitalTrends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapy rawling JSON-LD (DigitalTrends)

## Parte 1- Compilar datos de documentos web (0,75 puntos)
En el ejemplo siguiente definimos un crawler en Scrapy para extraer noticias de los metadatos de las páginas web.
DigitalTrends y otros blogs publican metadatos en formato JSON-LD (https://json-ld.org/) que permite obtener información estructurada de las webs. En este caso, tendremos que obtener esos objetos JSON-LD y extraer su información en el formato de noticia (https://schema.org/NewsArticle) publicado en Schema.org


In [ ]:
!pip3 install -U scrapy

In [22]:
!pip3 install stanza
import stanza
stanza.download('en')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 10.9 MB/s eta 0:00:00


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /stanfordnlp/stanza-resources/main/resources_1.7.0.json HTTP/1.1" 200 46043


INFO:stanza:Downloading default packages for language: en (English) ...
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /stanfordnlp/stanza-en/resolve/v1.7.0/models/default.zip HTTP/1.1" 302 1060
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn-lfs.huggingface.co:443
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /stanfordnlp/stanza-en/abb565cb9f6c2daf01cc91583425b9cf44134a1e0280218039a8b77f98e57155?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27default.zip%3B+filename%3D%22default.zip%22%3B&response-content-type=application%2Fzip&Expires=1702031330&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjAzMTMzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zdGFuZm9yZG5scC9zdGFuemEtZW4vYWJiNTY1Y2I5ZjZjMmRhZjAxY2M5MTU4MzQyNWI5Y2Y0NDEzNGExZTAyODAyMTgwMzlhOGI3N2Y5OGU1NzE1NT9yZXNwb25zZ

INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [2]:
import scrapy
import sys
import json
import locale
import time
import random

import hashlib

from bs4 import BeautifulSoup


# Para cada crawler que nos definimos nos debemos crear una clase Spider que debe heredar de la clase scrapy.Spider

class DigitalTrendsSpider (scrapy.Spider):
    name = 'DigitalTrends'

    # Decimos que el dominio válido es el de la UM
    allowed_domains = ['www.digitaltrends.com']

    # podemos definir las páginas de inicio
    start_urls = ['https://www.digitaltrends.com']

    # para evitar que el sitio te bloquee por usar scrapy es interesante cambiar el USER_AGENT
    # El user agent por defecto de Scrapy cuando hace una petición es
    # Scrapy/VERSION (+https://scrapy.org)
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    }

    nDocumentos = 0
    visited_urls = set()

    # debemos de implementar este método que se llamará para cada una de las páginas que se vayan analizando
    def parse (self, response):
        """
        @inherit

        @param self
        @param response
        """

        if self.nDocumentos >= 1500:
          return

        # Guardamos la URL del sitio que se está visitando
        url = str(response.request.url).strip()

        # Cogemos el contenido relevante y para eso debemos usar selectores CSS
        for article in response.css ('.b-page '):

            # Cogemos el contenido del título
            title = str (article.css ('.b-headline__title  ').get ()).strip()
            title = BeautifulSoup (title, 'html.parser').get_text().strip()

            # autor
            autor = str (article.css ('.author.url.fn ').get ()).strip()
            autor = BeautifulSoup (autor, 'html.parser').get_text().strip()

            # Date
            date = str (article.css ('.b-byline__time.date.dtreviewed').get ()).strip()
            date = BeautifulSoup (date, 'html.parser').get_text().strip()

            #Content
            content = "".join (article.css ('.b-content.b-single__content.h-article-content ').get ())
            content = BeautifulSoup (content, 'html.parser').get_text().strip().replace("\"","").replace("\n","")

            pipelineStanza = stanza.Pipeline(lang='es', processors='tokenize ,mwt, pos, lemma, depparse,ner')
            entityTypes = pipelineStanza(content)

            dictEntities = dict()

            for sentence in entityTypes.sentences:
              print(sentence)
              for word in sentence.words:
                if word.upos not in dictEntities:
                  dictEntities[word.upos] = []
                dictEntities[word.upos].append(word.text)

            data = {
                'url' : url,
                'title': title,
                'autor': autor,
                'date': date,
                'content': content,
                'localization' : dicEntities["LOC"],
                'miscelaneous' : dicEntities['MISC']

            }


            title_hash = hashlib.md5(title.encode()).hexdigest()

            filename = f"{title_hash}.json"

            # Guardamos el documento si tiene contenido y título
            if content and title:
                print ("-------------------------")
                print (url)
                print (title)
                print (autor)
                print (date)
                print (content)
                print ("-------------------------")
                self.nDocumentos = self.nDocumentos + 1
                with open ('digitalTrends/' + filename, 'w') as f:
                    json.dump (data, f, indent = 4)



        # Obtenemos todas las otros links de la página representados por la etiqueta <a>


        url_in_current_document = response.css ('a')

        for next_page in url_in_current_document:
            # Para limitar que solamente se parseen las noticias dentro de 'https://www.digitaltrends.com/computing/ o https://www.digitaltrends.com/mobile/'
            # obtenemos el atributo href de la etiqueta <a> y parseamos la página

            url_str = str(next_page.css('::attr(href)').get())


            if ("https://www.digitaltrends.com/computing/" in url_str) or ("/mobile/" in url_str) and url_str not in self.visited_urls and self.nDocumentos < 1500:
              self.visited_urls.add(url_str)
              yield response.follow (next_page, self.parse)

In [ ]:
import os
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Como se ha definido anteriormente en el RSSCrawler, los ficheros se van a almacenar en la carpeta "rss"
# Comprobamos que existe la carpeta y si no existe la creamos
if (not os.path.exists('digitalTrends')):
    os.mkdir('digitalTrends')

# Creamos el proceso con el RSSSpider
process.crawl(DigitalTrendsSpider)
# Ejecutamos el Crawler
process.start()

In [4]:
def contar_archivos_en_carpeta(ruta):
    try:
        # Lista todos los archivos en la carpeta
        archivos = os.listdir(ruta)

        # Filtra solo los archivos (no directorios)
        archivos = [archivo for archivo in archivos if os.path.isfile(os.path.join(ruta, archivo))]

        # Devuelve el número de archivos
        return len(archivos)
    except Exception as e:
        print(f"Error al contar archivos: {e}")
        return None

contar_archivos_en_carpeta("digitalTrends")

1500

Parte 2 – Buscador (0,75 puntos)


In [5]:
!pip install whoosh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 10.0 MB/s eta 0:00:00


In [6]:
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID, DATETIME
from whoosh.qparser import QueryParser
from datetime import datetime
from dateutil import parser
from whoosh.analysis import StemmingAnalyzer
import re


In [7]:
# Define el esquema para el índice de Whoosh
schema = Schema(
    url=ID(unique=True, analyzer = StemmingAnalyzer()),
    title=TEXT(stored=True,analyzer = StemmingAnalyzer()),
    author=TEXT(stored=True,analyzer = StemmingAnalyzer()),
    date=DATETIME(stored=True),
    content=TEXT(stored=True,analyzer = StemmingAnalyzer())
)

index_dir = "whooshIndex"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)
    ix = create_in(index_dir, schema)
else:
    ix = open_dir(index_dir)



In [8]:
# Agrega los documentos al índice
# Aquí deberías leer tus archivos y agregarlos al índice
# Utiliza el bloque 'with' para garantizar que el índice se cierre correctamente
with ix.writer() as writer:
# Recorre todos los archivos en el directorio digitalTrends
  for filename in os.listdir('digitalTrends'):
      if filename.endswith(".json"):  # Asegúrate de que sean archivos JSON
          file_path = os.path.join('digitalTrends', filename)
          with open(file_path, 'r', encoding='utf-8') as file:
              try:
                  file_data = json.load(file)
                  date_string_clean = re.sub(r'\s+\d+:\d+[A|P]M', '', file_data['date'])
                  if date_string_clean != "None" :
                    date_obj = datetime.strptime(date_string_clean, '%B %d, %Y')
                  else :
                    date_obj = None
                  # Agrega los documentos al índice
                  writer.add_document(
                     url=file_data['url'],
                     title=file_data['title'],
                     author=file_data['autor'],
                     date = date_obj,
                     content=file_data['content']
                  )
              except json.JSONDecodeError as e:
                  print(f"Error al procesar el archivo {filename}: {e}")



In [11]:
from whoosh.index import open_dir
from whoosh.qparser import QueryParser,FuzzyTermPlugin
from whoosh import scoring
from whoosh.spelling import QueryCorrector
from whoosh.highlight import Highlighter, ContextFragmenter
from whoosh.qparser.dateparse import DateParserPlugin
from whoosh.analysis import StemmingAnalyzer


# Función para realizar búsquedas
def buscar_en_archivos(queryWords="", queryDate="", sortedBy="date", directory='whooshIndex'):
    # Se abre el índice existente o se crea uno nuevo si no existe
    ix = open_dir(directory)

    # Se crea un QueryParser para buscar en los campos específicos
    with ix.searcher(weighting=scoring.BM25F()) as searcher:
        #Indicamos que el parser va a ser sobre el campo content
        parser = QueryParser("content", schema=ix.schema)
        #Añadimos los plugins necesarios para las funcionalidades descritas
        #Plugin valido para realizar las busquedas por fechas
        parser.add_plugin(DateParserPlugin())
        # Añadir Stemming al parser para búsqueda más flexible
        parser.add_plugin(FuzzyTermPlugin())
        # Se inicializa la consulta combinada con la búsqueda por palabras y fecha
        user_query = parser.parse(queryWords)  # Búsqueda por palabras
        if queryDate:  # Si se proporciona una fecha, se agrega a la consulta
            date_query = parser.parse(queryDate)
            user_query = user_query & date_query

       # Se corrige la consulta en caso de que esté mal realizada
        correction = searcher.correct_query(q=user_query,qstring=queryWords)
        if correction.query != user_query:
          print("Did you mean:", correction.string)
        # Se procede a realizar la busqueda
        search_results = searcher.search(correction.query, limit=None, sortedby=sortedBy)
        # Configuramos el fragmenter para resaltar fragmentos de texto
        fragmenter = ContextFragmenter(maxchars=200, surround=30)
        highlighter = Highlighter(fragmenter=fragmenter)
        # Mostramos los resultados de las noticias que se han encontrado según las condiciones del usuario
        for hit in search_results:
            print(f"Título: {hit['title']}")
            print(f"Autor: {hit['author']}")
            print(f"Fecha: {hit['date']}")
            print(f"Fragmento de contenido: {hit.highlights('content')}")
            print("--------------")


In [ ]:
# Llamar a la función para realizar las búsquedas
buscar_en_archivos(queryWords="game", queryDate="date:[2022-01-01 TO 2022-12-28]", sortedBy="author")


Parte 3 – Vectorización (0,5 puntos)

In [13]:
import os
import json

# Ruta al directorio 'digitalTrends'
directory_path = 'digitalTrends'

# Lista para almacenar el contenido de 'content' de cada archivo
content_list = []

# Recorre todos los archivos en el directorio 'digitalTrends'
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):  # Verifica si es un archivo JSON
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                file_data = json.load(file)
                # Obtén el campo 'content' de cada archivo y añádelo a la lista
                content = file_data.get('content', '')  # Si 'content' no existe, asigna una cadena vacía
                content_list.append(content)
            except json.JSONDecodeError as e:
                print(f"Error al procesar el archivo {filename}: {e}")

# Ahora content_list contiene el campo 'content' de cada archivo en 'digitalTrends'
# Puedes utilizar content_list para convertir el texto a vectores usando TfidfVectorizer o cualquier otro método



In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

# Crear un objeto TfidfVectorizer con opciones de preprocesamiento
tfidf_vectorizer = TfidfVectorizer(
    stop_words='english',  # Usar stopwords del inglés
    min_df=10  # Filtrar términos que aparecen en menos de 10 documentos
)


# Obtener la matriz documento-término
matriz_tfidf = tfidf_vectorizer.fit_transform(content_list)

# Obtener los términos más relevantes de toda la colección (los 10 términos con mayor TF-IDF)
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = matriz_tfidf.max(axis=0).toarray().flatten()
indices_top_10 = tfidf_scores.argsort()[-10:][::-1]
top_10_terminos = [feature_names[i] for i in indices_top_10]

feature_names_count = tfidf_vectorizer.get_feature_names_out()
term_frequencies = matriz_tfidf.sum(axis=0).A1
indices_top_100 = term_frequencies.argsort()[-100:][::-1]
top_100_repetidos = [feature_names_count[i] for i in indices_top_100]

print(matriz_tfidf)
# Mostrar resultados
print("Los 10 términos más relevantes de toda la colección (con mayor TF-IDF):")
print(top_10_terminos)
print("\nLos 100 términos más repetidos en la colección:")
print(top_100_repetidos)


  (0, 3957)	0.006900472699699308
  (0, 4836)	0.006869889071011471
  (0, 5746)	0.006547853270707017
  (0, 31)	0.005539691450953903
  (0, 5668)	0.015899587449679894
  (0, 4134)	0.008726766429177902
  (0, 2735)	0.011453033082401512
  (0, 675)	0.011663702569788499
  (0, 6011)	0.009877848850164395
  (0, 3287)	0.006052339320360289
  (0, 4966)	0.0056290437214364395
  (0, 1690)	0.006263665931307873
  (0, 4624)	0.002448749877927993
  (0, 2033)	0.002448749877927993
  (0, 6145)	0.003941730270455949
  (0, 1359)	0.010516589283697714
  (0, 4495)	0.010256929435919063
  (0, 1127)	0.005268856564155923
  (0, 1565)	0.011591423418931301
  (0, 951)	0.011738206040273114
  (0, 2594)	0.007583359964547306
  (0, 3550)	0.009741408712486846
  (0, 5785)	0.011322127053977143
  (0, 1763)	0.01975569770032879
  (0, 727)	0.008538008147840956
  :	:
  (1499, 4370)	0.05292902509667185
  (1499, 2983)	0.007752660976549857
  (1499, 1837)	0.009733228950593956
  (1499, 12)	0.014469928525707738
  (1499, 4398)	0.0066596909561065

PARTE 4: Detección de entidades

In [33]:
pipelineStanza = stanza.Pipeline(lang='es', processors='tokenize ,mwt, pos, lemma, depparse,ner')
entityTypes = pipelineStanza(content)
dictEntities = dict()
for sentence in entityTypes.sentences:
  print(sentence)
  for ent in sentence.ents:
    print(ent)


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /stanfordnlp/stanza-resources/main/resources_1.7.0.json HTTP/1.1" 200 46043


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | ancora          |
| mwt       | ancora          |
| pos       | ancora_charlm   |
| lemma     | ancora_nocharlm |
| depparse  | ancora_charlm   |
| ner       | conll02         |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[
  {
    "id": 1,
    "text": "Dan",
    "lemma": "Dan",
    "upos": "VERB",
    "xpos": "vmip3p0",
    "feats": "Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin",
    "head": 0,
    "deprel": "root",
    "start_char": 0,
    "end_char": 3,
    "ner": "B-MISC",
    "multi_ner": [
      "B-MISC"
    ]
  },
  {
    "id": 2,
    "text": "Baker/Digital",
    "lemma": "Baker/Digital",
    "upos": "PROPN",
    "head": 1,
    "deprel": "flat",
    "start_char": 4,
    "end_char": 17,
    "ner": "I-MISC",
    "multi_ner": [
      "I-MISC"
    ]
  },
  {
    "id": 3,
    "text": "TrendsRyzen",
    "lemma": "TrendsRyzen",
    "upos": "PROPN",
    "head": 1,
    "deprel": "flat",
    "start_char": 18,
    "end_char": 29,
    "ner": "I-MISC",
    "multi_ner": [
      "I-MISC"
    ]
  },
  {
    "id": 4,
    "text": "processors",
    "lemma": "processors",
    "upos": "PROPN",
    "head": 1,
    "deprel": "flat",
    "start_char": 30,
    "end_char": 40,
    "ner": "I-MISC",
    "multi_ner":